# Reduce a hypergraph and compute RBS homology



<span style="color:red">
Important notes: 
</span>

- the strategy is to represent the complex as the Vietoris Rips complex of the adjcency matrix of the complement of the (undirected) graph representing the edge containment poset
- there are more powerful reduction methods than the one shown here.  Enquire for details!

In [ ]:
import oat_python as oat

import copy
import plotly.graph_objects as go
import numpy as np
import networkx as nx
import hypernetx as hnx

# Define a hypergraph

In [ ]:
# define a hypergraph

E = { "A": ["x"], "B": ["y"], "C": ["x","y","z","zz"], "D": ["x","y","w","ww"], "DD": ["x","y","w","ww"] }

# Relabel and reduce

Homology calculations are only available for one hypergraphs in one data format: a list of sorted-lists of integers.  **However** we provide tools to translate back and forth between this and other data fromats, e.g. a dictionary of lists of strings.  

Below is one example.  Here we perform two actions in one step: 
- reduce the hypergraph by removing duplicate hyperedges (meaning hyperedges with the same vertex set) and duplicate nodes (meaning nodes that belong to the same set of hyperedges)
- reformat the reduced hypergraph as a list of sorted-lists of integers

In [ ]:
# collapse out redundant information, returning a list of lists
reduced_hg, label_translator = oat.hypergraph.reduce_hypergraph_with_labels(E)

# The label_translator object contains the information about what edge 
# (respectively, node) in the original hypergraph maps to which edge
# (respectively, node) in the reduced hypergraph.

# for key in label_translator.keys(): print(key)

print("\n\nReduced hypergraph edges")
print("------------------------")
display(reduced_hg)

print("\n\nNode / edge label translator")
print("----------------------------")
display(label_translator)

# Plot

### Initial hypergraph

In [ ]:
hnxgraph = hnx.Hypergraph(E)
hnx.drawing.draw(hnxgraph)

### Reduced hypergraph

In [ ]:
# plot the hypergraph

hnxgraph = hnx.Hypergraph(reduced_hg)
hnx.drawing.draw(hnxgraph)

### Reduced hypergraph, with original labels

Note that we have to
- label each node/vertex with a whole *set* of the old labels, since many nodes/edges in the original hypergraph can map to the the same node/edge of the reduced hypergraph
- here we converted each set of old labels to a string, to make it hashable; it's possible you can sidestep this step in HNX by using a different constructor

In [ ]:
remap_edge              =   lambda x: str(label_translator["new_edge_to_old_edges"][x])
remap_node              =   lambda x: str(label_translator["new_node_to_old_nodes"][x])
reduced_hg_relabeled    =   { remap_edge(edge_label): [ remap_node(n) for n in edge_nodes ] \
                                for edge_label, edge_nodes in enumerate(reduced_hg) }
hnxgraph                =   hnx.Hypergraph(reduced_hg_relabeled)
hnx.drawing.draw(hnxgraph)

# Compute homology

In [ ]:
# graph representing the edge containment poset (forgetting direction)
containment                 =   oat.hypergraph.edge_containment_graph_symmetrized( reduced_hg )
# graph whose edges form the set complement of the containment graph
anticontainment             =   nx.complement( containment )
# adjacency matrix of the anticontainment graph
anti_adjacency              =   nx.adjacency_matrix( anticontainment ).todense()
dissimilarity_matrtix       =   oat.dissimilarity.matrix_from_dense(
                                    dissimilarity_matrix    =   anti_adjacency,
                                    dissimilarity_max       =   0.5
                                )

# factored boundary matrix
factored                    =   oat.rust.FactoredBoundaryMatrixVr(
                                    dissimilarity_matrix    =   dissimilarity_matrtix,
                                    homology_dimension_max  =   1, 
                                )

In [ ]:
# print homology
homology        =   factored.homology(
                        return_cycle_representatives    =   True,
                        return_bounding_chains          =   True,
                    )
display(homology)

# Inspect a cycle representative

In [ ]:
cycle           =   homology["cycle representative"][1]
display(cycle)

### Relabel each vertex with an edge

Recall that each vertex in RBS homology represents an edge in the reduced hypergraph (which corresponds to a set of hyperedges in the initial hypergraph).  Here we relabel each vertex with **one** of the edges that maps to it.

In [ ]:

remap = { p: l[0] for p,l in enumerate(label_translator["new_edge_to_old_edges"]) }

cycle               =   copy.deepcopy(homology["cycle representative"][1])
cycle["simplex"]    =   [ [remap[x] for x in simplex] for simplex in cycle["simplex"] ]

cycle

### Relable each vertex with a **set** of edges

Recall that each vertex in RBS homology represents an edge in the reduced hypergraph (which corresponds to a set of hyperedges in the initial hypergraph).

In [ ]:
remap               =   label_translator["new_edge_to_old_edges"]

cycle               =   copy.deepcopy(homology["cycle representative"][1])
cycle["simplex"]    =   [ [remap[x] for x in simplex] for simplex in cycle["simplex"] ]
cycle


# Plot a cycle representative

In [ ]:
edges   =   homology["cycle representative"][1]["simplex"]
coo     =   oat.plot.hop_mds_from_simplices( edges.tolist() )

display(edges)
display(coo)

In [ ]:

remap               =   label_translator["new_edge_to_old_edges"]

data    =   []
for edge in edges:
    trace   =   oat.plot.edge__trace3d( edge=edge, coo=coo )
    label   =   "Simplex " + str( [ remap[x] for x in edge] )
    trace.update( name=label, text=label )
    data.append(trace)

fig = go.Figure(data)
fig.update_layout(title="Hover the cursor to show the label of each simplex!")
fig.show()

